# Super Mario Bros 
Es un entorno visual, la entrada es una imagen de múltiples frames por segundo, lo que se puede usar:

* CNN (Convolutional Neural Networks): Para extraer características espaciales de las imágenes.

* CNN + RNN (LSTM): Si quieres considerar la secuencia temporal (ej. para decisiones dependientes del pasado).

* DQN (Deep Q-Network): Para entornos con acción discreta como Mario.

* Double DQN o Dueling DQN: Para mejorar la estabilidad del entrenamiento.

* Rainbow DQN: Una combinación mejorada de múltiples técnicas DQN.

Para simplificar en TF-Agents, se probara usar DQN + CNN.

# Set up
Instalamos las bibliotecas esenciales para ejecutar el entorno `gym-super-mario-bros` con `TF-Agents`. Esto incluye soporte para visualización de videos (`xvfb`, `ffmpeg`), el entorno de Mario Bros, `gym`, y `tf-agents`. También fijamos la versión de `imageio` a 2.4.0 por compatibilidad con la función `imageio.mimsave`, usada más adelante para guardar los videos de las partidas.


In [ ]:
!sudo apt-get -q update -q
!sudo apt-get install -y -q xvfb ffmpeg freeglut3-dev -q
!pip install -q 'imageio==2.4.0' -q
!pip install -q pyvirtualdisplay -q
!pip install -q tf-agents[reverb] -q
!pip install -q pyglet -q
!pip install -q swig -q
!pip install -q gym[atari,box2d,accept-rom-license] -q #install gym and virtual display 
!pip install -q gym-super-mario-bros -q

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf
import tf_agents
from tf_agents.environments import suite_gym
from tf_agents.environments.wrappers import ActionRepeat
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.utils import common
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.policies import random_tf_policy
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies import epsilon_greedy_policy

from tf_agents.environments import gym_wrapper
from tf_agents.environments import tf_py_environment

import gym
from nes_py.wrappers import JoypadSpace
#from gym.wrappers import GrayScaleObservation, ResizeObservation, FrameStack
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import matplotlib.pyplot as plt

# To get smooth animations
import matplotlib.animation as animation
matplotlib.rc('animation', html='jshtml')

# Print versions of imported packages
print(f"imageio version: {imageio.__version__}")
print(f"pyvirtualdisplay version: {pyvirtualdisplay.__version__}")

# Print TensorFlow version separately
print(f"tensorflow version: {tf.__version__}")

# Print tf-agents version
try:
    print(f"tf_agents version: {tf_agents.__version__}")
except AttributeError:
    print("tf_gents version: not available")

# Print other versions
print(f"gym version: {gym.__version__}")
print(f"matplotlib version: {matplotlib.__version__}")
print(f"PIL version: {PIL.Image.__version__}")  # Use PIL.Image to get version

# Handle reverb
try:
    import pkg_resources
    reverb_version = pkg_resources.get_distribution("reverb").version
    print(f"reverb version: {reverb_version}")
except Exception:
    print("reverb version: not available")

# Handle gym-super-mario-bros
try:
    print(f"gym-super-mario-bros version: {gym_super_mario_bros.__version__}")
except AttributeError:
    print("gym-super-mario-bros version: not available")

RL Definitions
==============

**Environment** The world that an agent interacts with and learns from.

**Action** $a$ : How the Agent responds to the Environment. The set of
all possible Actions is called *action-space*.

**State** $s$ : The current characteristic of the Environment. The set
of all possible States the Environment can be in is called
*state-space*.

**Reward** $r$ : Reward is the key feedback from Environment to Agent.
It is what drives the Agent to learn and to change its future action. An
aggregation of rewards over multiple time steps is called **Return**.

**Optimal Action-Value function** $Q^*(s,a)$ : Gives the expected return
if you start in state $s$, take an arbitrary action $a$, and then for
each future time step take the action that maximizes returns. $Q$ can be
said to stand for the "quality" of the action in a state. We try to
approximate this function.


Environment
===========

Initialize Environment
----------------------

Usamos `gym_super_mario_bros` para crear el entorno de juego `SuperMarioBros-v2`. Luego lo envolvemos con `JoypadSpace` para simplificar el espacio de acciones. En este caso, usamos `COMPLEX_MOVEMENT`, que ofrece combinaciones de movimientos más ricas como correr y saltar a la vez. También imprimimos las distintas configuraciones de movimiento disponibles para explorar otras opciones más simples como `RIGHT_ONLY` o `SIMPLE_MOVEMENT`.

Finalmente, hacemos un `reset` y ejecutamos un paso aleatorio para renderizar una imagen del entorno como verificación visual.



In [ ]:
env = gym_super_mario_bros.make('SuperMarioBros-v2')
env = JoypadSpace(env, COMPLEX_MOVEMENT)
print(SIMPLE_MOVEMENT)
print(COMPLEX_MOVEMENT)
print(RIGHT_ONLY)
state = env.reset()
env.step(env._action_space.sample())
img = env.render(mode="rgb_array")
plt.figure(figsize=(4, 6))
plt.imshow(img)
plt.axis("off")
plt.show()

Preprocess Environment
======================

Definimos una función `create_environment()` que envuelve el entorno de `SuperMarioBros-v2` con varias transformaciones útiles para facilitar el entrenamiento del agente. Estas transformaciones incluyen:

- `JoypadSpace`: se usa `SIMPLE_MOVEMENT` para reducir la complejidad del espacio de acción.
- `GrayScaleObservation`: convierte las imágenes a escala de grises para reducir la dimensionalidad.
- `ResizeObservation`: redimensiona las imágenes a 84x84 píxeles, un estándar común en entornos de juegos.
- `FrameStack`: apila las últimas 4 observaciones, permitiendo que el agente tenga noción de movimiento.
- `GymWrapper`: adapta el entorno para que sea compatible con TF-Agents.

In [ ]:
def create_mario_environment(
    level='SuperMarioBros-v2',
    movement=SIMPLE_MOVEMENT,
    grayscale=True,
    resize=84,
    stack_frames=4
):
    env = gym_super_mario_bros.make(level)
    env = JoypadSpace(env, movement)

    if grayscale:
        env = gym.wrappers.GrayScaleObservation(env, keep_dim=True)
    if resize:
        env = gym.wrappers.ResizeObservation(env, resize)
    if stack_frames:
        env = gym.wrappers.FrameStack(env, stack_frames)

    env = gym_wrapper.GymWrapper(env)
    return env

In [ ]:
# Entrenamiento y evaluación con entorno más simple
train_py_env = create_mario_environment(level='SuperMarioBros-v2', movement=SIMPLE_MOVEMENT)
eval_py_env = create_mario_environment(level='SuperMarioBros-v2', movement=SIMPLE_MOVEMENT)

# Entrenamiento y evaluación con entorno más complejo (nivel específico y más acciones)
train_py_env1 = create_mario_environment(level='SuperMarioBros-1-1-v2', movement=COMPLEX_MOVEMENT)
eval_py_env1 = create_mario_environment(level='SuperMarioBros-1-1-v2', movement=COMPLEX_MOVEMENT)

### Conversión a entornos compatibles con TF-Agents

TF-Agents requiere que los entornos estén en formato `TFPyEnvironment` para interactuar correctamente con los agentes y las políticas. Por eso, convertimos los entornos de entrenamiento y evaluación previamente definidos.

Los entornos convertidos (`train_env`, `eval_env`) se usarán durante el proceso de entrenamiento, evaluación y generación de episodios.


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(
    create_mario_environment(level='SuperMarioBros-v2', movement=SIMPLE_MOVEMENT)
)

eval_env = tf_py_environment.TFPyEnvironment(
    create_mario_environment(level='SuperMarioBros-v2', movement=SIMPLE_MOVEMENT)
)

# Definición de la red Q y del agente DQN

Creamos una red Q convolucional (`QNetwork`) para que el agente pueda procesar imágenes del entorno y tomar decisiones. Las observaciones (imágenes) se normalizan a valores entre 0 y 1 para estabilizar el entrenamiento. La arquitectura está compuesta por tres capas convolucionales y una capa totalmente conectada con 512 unidades.

Utilizamos un optimizador `RMSProp`, recomendado para entornos de juegos con imágenes, y configuramos una política epsilon-greedy con `epsilon` que decae linealmente desde 1.0 hasta 0.01 en 10,000 pasos, para favorecer la exploración al inicio y la explotación conforme avanza el entrenamiento.

Finalmente, instanciamos el agente `DqnAgent`, con una función de pérdida basada en errores cuadráticos y una tasa de descuento (`gamma`) de 0.99.

| Capa        | Filtros | Tamaño kernel | Stride |
|-------------|---------|----------------|--------|
| Conv2D #1   | 32      | (8, 8)         | 4      |
| Conv2D #2   | 64      | (4, 4)         | 2      |
| Conv2D #3   | 64      | (3, 3)         | 1      |
| Dense       | 512     | -              | -      |


In [ ]:
# Definir la red Q
preprocessing_layer = tf.keras.layers.Lambda(lambda x: tf.cast(x, np.float32) / 255.)  # Se crea una capa de preprocesamiento para escalar las imágenes entre 0 y 1
conv_layer_params = [  # Se definen los parámetros para las capas convolucionales
    (32, (8, 8), 4),  # Se configura la primera capa convolucional (número de filtros, tamaño del kernel, paso)
    (64, (4, 4), 2),  # Se configura la segunda capa convolucional
    (64, (3, 3), 1),  # Se configura la tercera capa convolucional
]

fc_layer_params = [512]  # Se definen los parámetros para las capas totalmente conectadas 'neuronas'

# Crear la red Q
q_net = QNetwork(
    input_tensor_spec = train_env.observation_spec(),  # Se especifica la entrada del entorno de entrenamiento
    action_spec = train_env.action_spec(),  # Se especifican las acciones del entorno de entrenamiento
    preprocessing_layers = preprocessing_layer,  # Se asigna la capa de preprocesamiento
    conv_layer_params = conv_layer_params,  # Se asignan los parámetros de las capas convolucionales
    fc_layer_params = fc_layer_params  # Se asignan los parámetros de las capas totalmente conectadas
)


# Definir el optimizador
optimizer = tf.compat.v1.train.RMSPropOptimizer(  # Se define el optimizador RMSProp
    learning_rate = 2.5e-4,  # Se establece la tasa de aprendizaje
    decay = 0.95,  # Se configura el decaimiento del optimizador
    momentum = 0.0,  # Se establece el momentum del optimizador
    epsilon = 0.01,  # Se configura el epsilon para el optimizador
    centered = True  # Se utiliza la versión centrada del RMSProp
)
# Definir el contador global de pasos
train_step_counter = tf.Variable(0)

epsilon = tf.compat.v1.train.polynomial_decay(
    learning_rate=1.0,  # Valor inicial de epsilon
    global_step=train_step_counter,
    decay_steps=10000,  # Número de pasos para reducir epsilon
    end_learning_rate=0.01,  # Valor mínimo de epsilon
    power=1.0)  # Controla la tasa de decay (1.0 es lineal)

# Usar el decay en el agente
# Inicializar el agente DQN
agent = DqnAgent(
    time_step_spec = train_env.time_step_spec(),  # Se especifica el tiempo del entorno de entrenamiento
    action_spec = train_env.action_spec(),  # Se especifican las acciones del entorno de entrenamiento
    q_network = q_net,  # Se asigna la red Q que se utilizará
    optimizer = optimizer,  # Se asigna el optimizador para el agente
    td_errors_loss_fn = common.element_wise_squared_loss,  # Se define la función de pérdida para errores temporales
    train_step_counter = train_step_counter,  # Se asigna el contador de pasos de entrenamiento
    gamma = 0.99,  # Se establece el factor de descuento
    epsilon_greedy = epsilon,  # Se configura la probabilidad de elegir una acción aleatoria
    target_update_period = 10000  # Se establece la frecuencia para actualizar el objetivo
)


# Inicialización del agente y configuración del replay buffer

Inicializamos el agente `DqnAgent` con `agent.initialize()` para preparar todas sus variables internas.

Luego, creamos un `TFUniformReplayBuffer`, que se encargará de almacenar las transiciones observadas por el agente. Este buffer es esencial en el algoritmo DQN, ya que permite muestrear experiencias de manera aleatoria durante el entrenamiento, lo cual rompe la correlación temporal entre datos consecutivos y estabiliza el aprendizaje.

- `data_spec`: define la estructura de los datos que se almacenarán (observaciones, acciones, recompensas, etc.).
- `batch_size`: viene del entorno (`train_env`) y define cuántos entornos paralelos se están usando (normalmente 1 si no se usa vectorización).
- `max_length`: controla la cantidad máxima de transiciones que el buffer puede almacenar (aquí 100,000).



In [ ]:
agent._time_step_spec

In [ ]:
agent.initialize()  # Se inicializa el agente DQN

# Create the replay buffer
replay_buffer = TFUniformReplayBuffer(
    data_spec = agent.collect_data_spec,  # Se especifica la estructura de los datos que se recogerán
    batch_size = train_env.batch_size,  # Se establece el tamaño del lote para el buffer, acciones, recompensas y otros elementos relevantes.
    max_length = 100000  # Se define la longitud máxima del buffer de repetición, se actualiza
)

In [ ]:
print("Train env batch size:", train_env.batch_size)


# Recolección de experiencia inicial y preparación del dataset

Antes de entrenar al agente, necesitamos llenar el replay buffer con experiencias. Esto se hace utilizando una política aleatoria (`RandomTFPolicy`), lo cual permite explorar el entorno de forma uniforme sin sesgo. Aquí recolectamos 10,000 pasos.

Luego, convertimos el buffer en un conjunto de datos (`as_dataset`), que puede usarse durante el entrenamiento para muestrear secuencias de transiciones. Este dataset se preprocesa usando `prefetch()` para mejorar la eficiencia y evitar cuellos de botella en la carga de datos.


In [ ]:
# Función para recolectar experiencia
def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()  # Se obtiene el estado actual del entorno
    action_step = policy.action(time_step)  # Se calcula la acción a partir del estado actual
    next_time_step = environment.step(action_step.action)  # Se aplica la acción en el entorno y se obtiene el siguiente estado
    traj = trajectory.from_transition(time_step, action_step, next_time_step)  # Se crea una trayectoria a partir de la transición
    buffer.add_batch(traj)  # Se añade la trayectoria al buffer de experiencias

# Se recolectan datos iniciales con una política aleatoria
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())  # Se define una política aleatoria
initial_collect_steps = 10000  # Se establece el número de pasos iniciales de recolección
for _ in range(initial_collect_steps):
    collect_step(train_env, random_policy, replay_buffer)  # Se recolecta experiencia utilizando la política aleatoria

# Se prepara el conjunto de datos
dataset = replay_buffer.as_dataset(  # Se convierte el buffer en un conjunto de datos
    num_parallel_calls=3,  # Se establece el número de llamadas paralelas
    sample_batch_size=64,  # Se define el tamaño del lote de muestra
    num_steps=2  # Se especifica el número de pasos a considerar en cada muestra
).prefetch(3)  # Se pre-carga el conjunto de datos
iterator = iter(dataset)  # Se crea un iterador para el conjunto de datos

# Función para evaluar el desempeño del agente

La función `evaluate_agent` ejecuta el agente sobre el entorno de evaluación por un número dado de episodios (`num_episodes`) y devuelve la recompensa promedio obtenida. Durante cada episodio, el agente selecciona acciones usando su política actual hasta que el episodio termina. Se acumulan las recompensas recibidas en cada paso y finalmente se calcula el promedio.

Esta métrica nos permite medir el progreso y desempeño real del agente sin exploración (política explotativa).

In [ ]:
def evaluate_agent(agent, eval_env, num_episodes=1):
    total_reward = 0.0
    for episode in range(num_episodes):
        time_step = eval_env.reset()
        policy_state = agent.policy.get_initial_state(eval_env.batch_size)
        episode_reward = 0
        
        while not time_step.is_last():
            action_step = agent.policy.action(time_step, policy_state)
            time_step = eval_env.step(action_step.action)
            episode_reward += time_step.reward.numpy() #
            
        total_reward += episode_reward
    avg_reward = total_reward / num_episodes
    return avg_reward

# Entrenamiento del agente DQN

Ejecutamos un loop de entrenamiento que realiza lo siguiente en cada iteración:

- Recolecta nuevas experiencias utilizando la política actual del agente (`agent.collect_policy`).
- Extrae un batch de experiencias del replay buffer y realiza una actualización del agente usando estas muestras.
- Cada cierto número de iteraciones (`log_interval`), imprime la pérdida para monitorear el progreso.

Se define un directorio de checkpoint para guardar el modelo periódicamente (comentado por ahora).

Se planea evaluar el agente cada `eval_interval` iteraciones, aunque esta parte está comentada y se puede habilitar para monitorear rendimiento.


In [ ]:
# Entrenamiento del agente
num_iterations = 145450  # Se ajusta este valor según los límites computacionales de Kaggle 350000 -> 145500
collect_steps_per_iteration = 1  # Se define el número de pasos de recolección por iteración
log_interval = 500  # Se establece el intervalo para los registros
eval_interval = 30000  # Evaluar el agente cada 1000 iteraciones

train_losses = []
eval_rewards = []
eval_iterations = []  # Lista para guardar las iteraciones en las que evalúas

checkpoint_dir = 'checkpoints/'
checkpoint = tf.train.Checkpoint(agent=agent)

# Guardar el modelo cada X iteraciones
save_interval = 40000  # Guardar cada 5000 iteraciones

for iteration in range(num_iterations):
    # Recolectar experiencia
    for _ in range(collect_steps_per_iteration):
        collect_step(train_env, agent.collect_policy, replay_buffer)  # Se llama a la función para recolectar experiencias

    # Muestra una experiencia del buffer y entrena al agente
    experience, _ = next(iterator)  # Se extrae un lote de experiencia del buffer
    train_loss = agent.train(experience).loss  # Se entrena al agente y se obtiene la pérdida
    
    if iteration % log_interval == 0:  # para cada intervalo
        print(f'Iteración: {iteration}, Pérdida: {train_loss}')  # Se imprime la iteración y la pérdida
        train_losses.append(train_loss.numpy())
        
    if iteration % eval_interval == 0:
        avg_reward = evaluate_agent(agent, eval_env, num_episodes=5)  # Mejor hacer varios episodios
        print(f'Evaluación en iteración {iteration}: Recompensa media = {avg_reward}')
        eval_rewards.append(avg_reward)
        eval_iterations.append(iteration)  # Guardar iteración actual

    if iteration % save_interval == 0:
        checkpoint.save(file_prefix=checkpoint_dir)
        print(f'Modelo guardado en la iteración {iteration}')


# Visualización del agente en acción

Configuramos una pantalla virtual para renderizar el entorno sin necesidad de una ventana gráfica visible, lo cual es útil en entornos remotos como Kaggle.

Definimos funciones para:

- Capturar los frames del entorno mientras el agente ejecuta su política (`run_and_visualize`).
- Crear una animación a partir de esos frames (`plot_animation`).

Finalmente, mostramos la animación directamente en el notebook para observar el comportamiento del agente entrenado.


In [ ]:
# Configurar una pantalla virtual para renderizar entornos de OpenAI Gym
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()  # Se inicia la pantalla virtual

# Funciones de visualización proporcionadas
def update_scene(num, frames, patch):  # Se define la función para actualizar la escena
    patch.set_data(frames[num])  # Se actualizan los datos del frame actual
    return patch  # Se devuelve el parche actualizado

def plot_animation(frames, repeat=False, interval=40):  # Se define la función para crear la animación
    fig = plt.figure()  # Se crea una nueva figura
    patch = plt.imshow(frames[0])  # Se muestra el primer frame
    plt.axis('off')  # Se ocultan los ejes
    anim = animation.FuncAnimation(  # Se crea la animación
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()  # Se cierra la figura para no mostrarla dos veces
    return anim  # Se devuelve la animación creada

# Recoger frames para la visualización
def run_and_visualize(agent, env):
    frames = []
    time_step = env.reset()
    policy_state = agent.policy.get_initial_state(env.batch_size)
    while not time_step.is_last():
        action_step = agent.policy.action(time_step, policy_state)
        policy_state = action_step.state
        time_step = env.step(action_step.action)
        frame = np.squeeze(env.render())
         if frame.shape[-1] == 1:
            frame = np.repeat(frame, 3, axis=-1)
        frames.append(np.squeeze(frame))
    print(f"Total frames collected: {len(frames)}")
    return frames

# Ejecutar el agente y recoger marcos
frames1 = run_and_visualize(agent, eval_env)  # Se ejecuta la función para recoger marcos

# Crear y mostrar la animación
anim1 = plot_animation(frames1)  # Se crea la animación a partir de los marcos

# Mostrar la animación en Jupyter Notebook
from IPython.display import HTML  # Se importa la librería necesaria
HTML(anim1.to_jshtml())  # Se convierte la animación a formato HTML y se muestra

# Visualización de la política aleatoria

Definimos una política simple que elige acciones al azar entre las posibles del entorno.

Implementamos la función `run_and_visualize1` que ejecuta esta política aleatoria durante un número fijo de pasos (8000) en el entorno de evaluación, capturando los frames para luego generar una animación.

Esto nos permite comparar visualmente el comportamiento del agente entrenado versus un agente que actúa sin aprendizaje.


In [ ]:
train_env1 = tf_py_environment.TFPyEnvironment(train_py_env1)
eval_env1 = tf_py_environment.TFPyEnvironment(eval_py_env1)

# Configurar una pantalla virtual para renderizar entornos de OpenAI Gym
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()  # Se inicia la pantalla virtual

# Funciones de visualización proporcionadas
def update_scene(num, frames, patch):  # Se define la función para actualizar la escena
    patch.set_data(frames[num])  # Se actualizan los datos del frame actual
    return patch  # Se devuelve el parche actualizado

def plot_animation1(frames, repeat=False, interval=40):  # Se define la función para crear la animación
    fig = plt.figure()  # Se crea una nueva figura
    patch = plt.imshow(frames[0])  # Se muestra el primer frame
    plt.axis('off')  # Se ocultan los ejes
    anim = animation.FuncAnimation(  # Se crea la animación
        fig, update_scene, fargs=(frames, patch),
        frames=len(frames), repeat=repeat, interval=interval)
    plt.close()  # Se cierra la figura para no mostrarla dos veces
    return anim  # Se devuelve la animación creada
def random_policy():
    return np.random.choice(len(SIMPLE_MOVEMENT))
# Recoger frames para la visualización
def run_and_visualize1(env):
    frames = []
    time_step = env.reset()
    
    for i in range(8000):
        action_step = random_policy()
        time_step = env.step(action_step)
        
        # Obtener el frame y asegurarse de que es un array numpy
        frame = np.squeeze(env.render())
               
        frames.append(frame)
        i += 1

    print(f"Total frames collected: {len(frames)}")
    return frames


# Ejecutar el agente y recoger marcos
frames_random = run_and_visualize1(eval_env1)  # Se ejecuta la función para recoger marcos

# Crear y mostrar la animación
anim = plot_animation1(frames_random)  # Se crea la animación a partir de los marcos

# Mostrar la animación en Jupyter Notebook
from IPython.display import HTML  # Se importa la librería necesaria
HTML(anim.to_jshtml())  # Se convierte la animación a formato HTML y se muestra

# Gráfica de recompensa promedio

Esta gráfica muestra cómo la recompensa promedio obtenida por el agente en el entorno de evaluación evoluciona a lo largo de las iteraciones de entrenamiento. Un aumento en esta métrica indica que el agente está aprendiendo a tomar mejores decisiones para maximizar la recompensa acumulada.


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(eval_iterations, eval_rewards, marker='o', linestyle='-')
plt.title('Evolución de la recompensa promedio durante el entrenamiento')
plt.xlabel('Iteraciones')
plt.ylabel('Recompensa promedio')
plt.grid(True)
plt.show()